# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Import WeatherPy CSV
weather_csv = pd.read_csv("resources/city_weather_data.csv")

weather_csv.dropna(inplace =True)
weather_csv.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qaqortoq,60.72,-46.03,42.80,41,0,2.24,GL,1601236435
1,1,Kahului,20.89,-156.47,84.20,54,20,13.87,US,1601236435
2,2,Bucerías,20.75,-105.33,89.01,70,20,9.17,MX,1601236436
3,3,Mitsamiouli,-11.38,43.28,76.69,77,11,14.07,KM,1601236436
4,4,Sørland,67.67,12.69,46.40,52,9,5.82,NO,1601236436


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store location and weight 
location = weather_csv[["Lat", "Lng"]].astype(float)
weight = weather_csv["Humidity"].astype(float)

In [19]:
# Adding heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [57]:
# Creating DataFrame for my criteria
my_criteria = weather_csv.loc[(weather_csv["Max Temp"].astype(float) > 75) &
                              (weather_csv["Humidity"].astype(float) <= 60) &
                              (weather_csv["Cloudiness"].astype(float) <= 40)]
 
my_criteria = my_criteria.drop(columns="Unnamed: 0")
my_criteria.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Kahului,20.89,-156.47,84.20,54,20,13.87,US,1601236435
9,Saint George,37.10,-113.58,87.01,23,1,8.05,US,1601236246
16,Adrar,20.50,-10.07,96.48,14,0,12.95,MR,1601236439
19,Lachi,33.38,71.34,79.45,38,0,3.20,PK,1601236440
21,Tessalit,20.20,1.01,90.27,11,0,3.94,ML,1601236440


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
hotel_df = my_criteria
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Kahului,20.89,-156.47,84.20,54,20,13.87,US,1601236435,
9,Saint George,37.10,-113.58,87.01,23,1,8.05,US,1601236246,
16,Adrar,20.50,-10.07,96.48,14,0,12.95,MR,1601236439,
19,Lachi,33.38,71.34,79.45,38,0,3.20,PK,1601236440,
21,Tessalit,20.20,1.01,90.27,11,0,3.94,ML,1601236440,


In [61]:

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
# get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"  

#  assemble url 
    response = requests.get(url, params=params).json()
# extract results
    results = response["results"]
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
      


Missing field/result... skipping.
Missing field/result... skipping.


In [62]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,Kahului,20.89,-156.47,84.20,54,20,13.87,US,1601236435,Kahului
9,Saint George,37.10,-113.58,87.01,23,1,8.05,US,1601236246,St. George
16,Adrar,20.50,-10.07,96.48,14,0,12.95,MR,1601236439,
19,Lachi,33.38,71.34,79.45,38,0,3.20,PK,1601236440,Lachi
21,Tessalit,20.20,1.01,90.27,11,0,3.94,ML,1601236440,Achemelmel


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=weight, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)
fig.add_layer(heat_layer)

# Create a marker_layer using the hotel name, city and country

marker_layer = gmaps.marker_layer(locations,
    info_box_content=hotel_info)


fig.add_layer(marker_layer)

# Display Map
fig


Figure(layout=FigureLayout(height='420px'))